# River Flow Dynamics

Cada gota representa una solución al problema.
Entorno: grafo del problema del viajante simétrico a resolver.
Se representa como una matriz de adyacencia en la que el valor de la posición [i][j] es la distancia entre las ciudades i y j.

In [ ]:
import random
import copy
import json
import time
import tsplib95

### Función cargaCasoDePrueba

Lee un caso de prueba del fichero con nombre _nombreFichero_ y devuelve la matriz de adyacencia correspondiente.

In [ ]:
def cargaCasoDePrueba(nombreFichero):
    with open(nombreFichero, 'r') as archivo:
        matrizAdy = json.loads(archivo.read())
    return matrizAdy

### Función tsplibAMatrizAdyacencia

Convierte el problema dado del formato tsplib a la matriz de adyacencia correspondiente.

In [ ]:
def tsplibAMatrizAdyacencia(problema):
    matrizAdy = [[]] * problema.dimension
    for i in range(problema.dimension):
        matrizAdy[i] = [problema.get_weight(i+1,j+1) for j in range(problema.dimension)]
    return matrizAdy

### Función cargaCasoTsplib

Carga un problema de la tsplib y lo devuelve ya en forma de matriz de adyacencia.

In [ ]:
def cargaCasoTsplib(nombreFichero):
    problema = tsplib95.load(nombreFichero)
    return tsplibAMatrizAdyacencia(problema)

### Función guardarResultados

Dadas las ejecuciones de un algoritmo, las guarda en un fichero formateado adecuadamente para posteriormente cargarlas en el notebook Graficas.ipynb

In [ ]:
def guardarResultados(ejecuciones, problema, algoritmo, num):
    nombreFichero = problema + "_" + algoritmo + "_" + str(num) + ".txt"
    with open(nombreFichero, 'a') as archivo:
        for ejecucion in ejecuciones:
            archivo.write(str(ejecucion[0]) + '\n')
            for hormiga in ejecucion[1]:
                archivo.write(str(hormiga))
            archivo.write('FIN\n')

## Arista
La clase Arista nos permite representar un camino entre dos ciudades concretas. Cada arista lleva una altura asociada que actúa como barrera según se propone en la adaptación al TSP del algoritmo de RFD.

In [ ]:
class Arista:
    def __init__(self, desde, hasta, distancia=None):
        self.desde = desde
        self.hasta = hasta
        self.distancia = 1 if distancia is None else distancia
        self.altura = 10000

## Entorno
La clase Entorno representa una instancia del problema del viajante a resolver. Contiene una serie determinada de Aristas que crean el grafo a resolver. Además también guarda la altura de los nodos en cada momento. El nodo 0 es del que partirán todas nuestras soluciones (recordemos que en el TSP lo importante es el orden en el que se visitan las ciudades y no por cuál se empiece). Por ello, lo trataremos de manera especial como origen y destino a la vez mediante un nodo auxiliar.

In [ ]:
class Entorno:
    def __init__(self, matrizAdy):
        self.conjuntoAristas = set()
        self.aristas = self.creaAristas(matrizAdy)
        self.numVertices = len(matrizAdy)
        # Inicializamos las alturas de los nodos a 10000.
        # El nodo de destino adicional se gestionará como un caso aparte
        # ya que siempre va a ser el último en visitarse. Tendrá altura 0.
        self.alturas = [10000 for i in range(self.numVertices)]
        # Probabilidad de que una gota suba una arista a mayor altura.
        self.prob_remontar = 0.1
        
    def creaAristas(self, matrizAdy):
        aristas = {}
        for i in range(len(matrizAdy)):
            for j in range(i):
                arista = Arista(i, j, matrizAdy[i][j])
                aristas[i,j] = arista
                aristas[j,i] = arista
                self.conjuntoAristas.add(arista)
        return aristas
    
    def resetearAlturas(self):
        self.alturas = [10000 for i in range(self.numVertices)]
        for arista in self.conjuntoAristas:
            arista.altura = 10000
            if arista.hasta == 0:
                arista.altura = 0

## Gota
La clase Gota representa una solución concreta al problema planteado en el entorno.
Realiza un trayecto concreto entre las ciudades eligiendo sus movimientos en base a la diferencia de altura entre las mismas.

In [ ]:
class Gota:

    def __init__(self, entorno):
        self.nodoActual = None
        self.inicio = None
        self.entorno = entorno
        self.visitados = []
        self.porVisitar = []
        self.trayecto = []
        self.distancia = 0
        # Atascada será true cuando la gota no tenga ningún sitio al que bajar.
        self.atascada = False
        
    def __str__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
    
    def __repr__(self):
        resultado = str(self.distancia) + '\n' + str(self.visitados) + '\n'
        return resultado
        
    # Nos va a interesar poder ordenar las gotas según la distancia de su solución
    # para poder ordenarlas y quedarnos con la mejor.
    def __eq__(self,other):
        return self.distancia == other.distancia
    
    def __lt__(self,other):
        return self.distancia < other.distancia
        
    def inicializar(self, inicio):
        self.inicio = inicio
        self.nodoActual = inicio
        self.visitados = [inicio]
        self.porVisitar = [v for v in range(entorno.numVertices) if v != inicio]
        self.trayecto = []
        self.distancia = 0
        self.atascada = False
        return self
    
    # Elige un movimiento y lo efectúa devolviendo la arista elegida.
    def movimiento(self):
        eleccion = self.elegirMovimiento()
        # Caso en el que la gota se atasca
        if eleccion is None: return None
        return self.realizaMovimiento(eleccion)
    
    # Elige el movimiento a realizar. Devuelve la arista a usar.
    def elegirMovimiento(self):
        # Sólo queda volver al nodo original
        if not self.porVisitar:
            return self.entorno.aristas[self.nodoActual,self.inicio]
        listaAristas = []
        puedeSubir = random.random() <= self.entorno.prob_remontar
        for nodo in self.porVisitar:
            if self.entorno.aristas[self.nodoActual,nodo].altura < self.entorno.alturas[self.nodoActual] or puedeSubir:
                listaAristas.append(self.entorno.aristas[self.nodoActual,nodo])
        if not listaAristas:
            self.atascada = True
            return None
        listaGradientes = [self.calculaGradienteDesde(arista) for arista in listaAristas]
        gradienteAcumulado = sum(listaGradientes)
        probabilidades = []
        for gradiente in listaGradientes:
            # Calculamos las probabilidades en %
            probabilidades.append(100 * gradiente/gradienteAcumulado)
        return random.choices(listaAristas, weights=probabilidades)[0]
    
    def calculaGradienteDesde(self, arista):
        desde = arista.desde if self.nodoActual == arista.desde else arista.hasta
        # En el caso en el que sea una arista plana devolvemos un valor pequeño para que la gota se pueda desplazar
        diferencia = entorno.alturas[desde] - arista.altura
        if diferencia == 0: return 0.1 / arista.distancia
        elif diferencia < 0: return 0.1 / abs(diferencia)
        return diferencia / arista.distancia
    
    def calculaGradienteHasta(self, arista):
        hasta = arista.hasta if self.nodoActual == arista.desde else arista.desde
        # En el caso en el que sea una arista plana devolvemos un valor pequeño para que la gota se pueda desplazar
        diferencia = arista.altura - entorno.alturas[hasta]
        if diferencia == 0: return 0.1 / arista.distancia
        elif diferencia < 0: return 0.1 / abs(diferencia)
        return diferencia / arista.distancia
    
    def realizaMovimiento(self, arista):
        hasta = arista.hasta if self.nodoActual == arista.desde else arista.desde
        if  not (hasta == self.inicio):
            self.visitados.append(hasta)
            self.porVisitar.remove(hasta)
        self.distancia += arista.distancia
        self.nodoActual = hasta
        return arista
        

## Funciones

### Función crearGotas

Esta función genera un nuevo grupo de gotas y las inicializa en la posición inicial del grafo.

In [ ]:
def crearGotas(entorno, numGotas):
    gotas = []
    for i in range(numGotas):
            nuevaGota = Gota(entorno).inicializar(0)
            gotas.append(nuevaGota)
    return gotas

### Función reiniciarGotas

Esta función reubica las gotas en el punto inicial del grafo y las prepara para una nueva ejecución.

In [ ]:
def reiniciarGotas(gotas):
    for gota in gotas:
        gota.inicializar(0)

### Función encontrarSoluciones

Realiza todos los movimientos de todas las gotas según lo indicado en River Flow Dynamics. Gestiona también el caso en el que las gotas se quedan atascadas.

In [ ]:
# Dos bucles: ya que todas las gotas tienen que moverse por los nodos hasta haberlos visitado todos.
def encontrarSoluciones(entorno, gotas):
    for i in range(entorno.numVertices):
        for gota in gotas:
            if not gota.atascada:
                arista = gota.movimiento()
                if arista is not None:
                    gota.trayecto.append(arista)
    gotas = sorted(gotas)
    for gota in gotas:
        if not gota.atascada:
            return gota
    return None

### Función realizaErosion

Erosiona los nodos según el resultado de la iteración del algoritmo.

In [ ]:
def realizaErosion(entorno, gotas):
    for gota in gotas:
        if not gota.atascada:
            for arista in gota.trayecto:
                desde = arista.desde if gota.nodoActual == arista.desde else arista.hasta
                hasta = arista.hasta if gota.nodoActual == arista.desde else arista.desde
                if hasta != 0:
                    erosion = gota.calculaGradienteDesde(arista)
                    entorno.alturas[desde] = max(entorno.alturas[desde] - erosion, 0)
                    erosion = gota.calculaGradienteHasta(arista)
                    arista.altura = max(arista.altura - entorno.alturas[hasta], 0)

### Función depositarSedimentos

Deposita sedimentos en cada nodo y arista. Deposita sedimentos adicionales en los nodos de las gotas atascadas.

In [ ]:
def depositarSedimentos(entorno, gotas):
    entorno.alturas = [altura + 10000 for altura in entorno.alturas]
    for arista in entorno.conjuntoAristas:
        if arista.hasta != 0:
            arista.altura = arista.altura + 10000
    for gota in gotas:
        if gota.atascada:
            entorno.alturas[gota.nodoActual] = entorno.alturas[gota.nodoActual] + 100

### Funciones RFD

Dado un entorno, un número de iteraciones, y un número de gotas, aplica la metaheurística de River Flow Dynamics para intentar obtener una buena solución al problema del TSP. Devuelve la mejor solución encontrada tras todas las iteraciones. Cada una usa una condición de parada diferente.

In [ ]:
def RFDTiempo(entorno, numGotas):
    tiempoInicio = time.time()
    listaSoluciones = []
    entorno.resetearAlturas()
    mejorSolucionGlobal = None
    gotas = crearGotas(entorno, numGotas)
    # Paramos el algoritmo tras 300 segundos (5 minutos)
    while time.time() - tiempoInicio < 300:
        reiniciarGotas(gotas)
        mejorSolucionActual = encontrarSoluciones(entorno, gotas)
        if mejorSolucionActual is not None:
            if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
                mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
        if mejorSolucionGlobal is not None:
            listaSoluciones.append(mejorSolucionGlobal)
        realizaErosion(entorno, gotas)
        depositarSedimentos(entorno, gotas)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

In [ ]:
def RFD(entorno, numGotas):
    tiempoInicio = time.time()
    # Si no se mejora la solución después de maxSeguidas iteraciones, se para el algoritmo.
    seguidas = 0
    maxSeguidas = 10000
    listaSoluciones = []
    entorno.resetearAlturas()
    mejorSolucionGlobal = None
    gotas = crearGotas(entorno, numGotas)
    while seguidas < maxSeguidas:
        reiniciarGotas(gotas)
        mejorSolucionActual = encontrarSoluciones(entorno, gotas)
        if mejorSolucionActual is not None:
            if mejorSolucionGlobal is None or mejorSolucionActual < mejorSolucionGlobal:
                seguidas = 0
                mejorSolucionGlobal = copy.deepcopy(mejorSolucionActual)
            else:
                seguidas += 1
        else:
            seguidas += 1
        if mejorSolucionGlobal is not None:
            listaSoluciones.append(mejorSolucionGlobal)
        realizaErosion(entorno, gotas)
        depositarSedimentos(entorno, gotas)
    tiempoFin = time.time()
    tiempo = tiempoFin - tiempoInicio
    return listaSoluciones, tiempo

## Pruebas

In [ ]:
# Cambiar por el nombre del problema deseado
nombreProblema = 'eil51'
matrizAdy = cargaCasoTsplib('ALL_tsp/' + nombreProblema + '.tsp')
# Alterar este range si se quiere ejecutar pruebas con varios números de individuos.
# Por defecto, se empieza por un número de individuos igual al número de ciudades,
# y se va aumentando de 5 en 5 hasta 5 veces esa cifra.
for numGotas in range(len(matrizAdy), 5 * len(matrizAdy), 5):
    # 20 ejecuciones con cada nº de gotas
    ejecuciones = []
    for i in range(20):
        entorno = Entorno(matrizAdy)
        soluciones, tiempo = RFD(entorno, numGotas)
        ejecuciones.append((tiempo, soluciones))
    guardarResultados(ejecuciones, nombreProblema, 'RFD', numGotas)